# 1. Импортируем библиотеки и датасет 

In [1]:
import pandas as pd
import random
import numpy as np
import re
import nltk
import nltk.corpus
import string
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/elizabethsolomyannik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/elizabethsolomyannik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
tsv_file = 'reviews.tsv'
df_all = pd.read_table(tsv_file, sep='\t')
df = df_all.sample(n = 10000)
df.to_csv('new_name.csv', index=False, encoding="utf8")

In [3]:
df.head()

,permalink,rating,review_text
41298,1165596568,5.0,Ризотто просто изумительное!
4994,1017954085,4.0,Проблема с парковкой. Стулья на веранде сырые.
159745,54766500180,5.0,Мне нравится.
49993,1214095315,5.0,Поскольку в этот раз мы приехали с супругой в ...
7704,1026518464,5.0,"Очень приветливый персонал, все вкусно, как дома."


# 2.1 Чистим датасет от пропущенных значений в поле rating и от rating = 0.0

In [4]:
len(df) #кол-во строк в датасете

10000

In [5]:
df = df.loc[df['rating'] != 0.0] #оставляем строки со значением 'rating' не 0.0

In [6]:
df.head()

,permalink,rating,review_text
41298,1165596568,5.0,Ризотто просто изумительное!
4994,1017954085,4.0,Проблема с парковкой. Стулья на веранде сырые.
159745,54766500180,5.0,Мне нравится.
49993,1214095315,5.0,Поскольку в этот раз мы приехали с супругой в ...
7704,1026518464,5.0,"Очень приветливый персонал, все вкусно, как дома."


In [7]:
len(df) #кол-во строк в датасете без 0.0 

9995

In [8]:
df = df.dropna() #удаляем строки, в которых есть пропущенные значения столбцов

In [9]:
len(df)

9989

In [10]:
df.head()

,permalink,rating,review_text
41298,1165596568,5.0,Ризотто просто изумительное!
4994,1017954085,4.0,Проблема с парковкой. Стулья на веранде сырые.
159745,54766500180,5.0,Мне нравится.
49993,1214095315,5.0,Поскольку в этот раз мы приехали с супругой в ...
7704,1026518464,5.0,"Очень приветливый персонал, все вкусно, как дома."


# 2.2 Очистка текста от html тэгов и знаков препинания

In [11]:
df['review_text'] = df['review_text'].astype(str) #переводим столбец в строку

In [12]:
df['review_text'] = df['review_text'].str.replace(r'\W', " ") #удаляем html тэги и пунктуацию c помощью регулярного выражения

In [13]:
df.head(50)

,permalink,rating,review_text
41298,1165596568,5.0,Ризотто просто изумительное
4994,1017954085,4.0,Проблема с парковкой Стулья на веранде сырые
159745,54766500180,5.0,Мне нравится
49993,1214095315,5.0,Поскольку в этот раз мы приехали с супругой в ...
7704,1026518464,5.0,Очень приветливый персонал все вкусно как дома
25541,1097028035,4.0,Тут можно самому жарить мясо А еще тут шикарн...
78717,1543719822,5.0,Простая и вкусная вегетарианская еда Уютно
120888,14489315917,5.0,Великолепный кофе думаю лучший в нашем город...
259861,165790668644,3.0,Хорошее место цены высокаты
54628,1243478327,5.0,Очень шикарный ресторан с прекрасной отмосферо...


# 2.3 Токенизация

In [14]:
from nltk.tokenize import word_tokenize

In [15]:
df['review_text'] = df['review_text'].apply(word_tokenize)

In [16]:
df.head()

,permalink,rating,review_text
41298,1165596568,5.0,"[Ризотто, просто, изумительное]"
4994,1017954085,4.0,"[Проблема, с, парковкой, Стулья, на, веранде, ..."
159745,54766500180,5.0,"[Мне, нравится]"
49993,1214095315,5.0,"[Поскольку, в, этот, раз, мы, приехали, с, суп..."
7704,1026518464,5.0,"[Очень, приветливый, персонал, все, вкусно, ка..."


# 2.4 Лемматизация и удаление стоп-слов

In [17]:
import pymystem3
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/elizabethsolomyannik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
from nltk.corpus import stopwords
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [19]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

def preprocess_text(text):
    tokens = []              #создаем пустой массив токенов
    for token in text:
        tokens.append(mystem.lemmatize(token.lower())[0])   
        tokens = [token for token in tokens if token not in russian_stopwords
              and token != " "\
              and token.strip() not in punctuation] 
        
    text = " ".join(tokens) #....................
    
    return text

In [20]:
for index, row in df.iterrows():
    df.loc[index, 'review_text'] = preprocess_text(row['review_text'])
    #row['review_text'] = preprocess_text(row['review_text'])

In [21]:
df.head()

,permalink,rating,review_text
41298,1165596568,5.0,ризотто просто изумительный
4994,1017954085,4.0,проблема парковка стул веранда сырой
159745,54766500180,5.0,нравиться
49993,1214095315,5.0,поскольку приезжать супруга санкт петербург вр...
7704,1026518464,5.0,очень приветливый персонал вкусно дома


# 3.1 Разбиваем датасет на тестовую и обучающую выборки

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X = df['review_text']
y = df['rating']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [25]:
len(X_train)

7991

In [26]:
len(X_test)

1998

In [27]:
X_train.head()

236926    отмечать корпоратив 19 12 2019 понравиться зап...
185465    стильный место кухня радовать качество доступн...
286769              вкусно сытно хороший ассортимент дорого
20086     вкусно достойный обслуживание ценник высоко ср...
124090                               душевный местечко ялта
Name: review_text, dtype: object

In [28]:
y_test.head()

253705    5.0
264797    5.0
112085    5.0
21085     5.0
40418     2.0
Name: rating, dtype: float64

# 4 Векторизация

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
X_train = X_train.to_frame()
X_test = X_test.to_frame()

In [31]:
X_train_words = X_train.apply(lambda x: x.str.split(expand=True).stack()).stack().value_counts()

In [32]:
X_train_dict = X_train_words.to_dict()

In [33]:
X_train_words.head()

очень      3106
вкусный    2397
вкусно     1851
хороший    1533
место      1474
dtype: int64

In [34]:
#corpus = df['review_text']
vectorizer = CountVectorizer()
X_train_review_bow = vectorizer.fit_transform(X_train['review_text'])
X_test_review_bow = vectorizer.transform(X_test['review_text'])

print('X_train_review_bow: ', X_train_review_bow.shape)
print('X_test_review_bow: ', X_test_review_bow.shape)

X_train_review_bow:  (7991, 10255)
X_test_review_bow:  (1998, 10255)


# Обучение модели

In [40]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

clf = MultinomialNB()
clf.fit(X_train_review_bow, y_train)

y_pred = clf.predict(X_test_review_bow) #вероятность ошибки
print('Test Accuracy: ', accuracy_score(y_test, y_pred)) 

Test Accuracy:  0.7857857857857858
